In [1]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\ETL_Project'

In [2]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

## Import Dependencies

In [3]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [4]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [5]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [6]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [7]:
class_data = pd.read_sql("SELECT * FROM class", conn)
class_data.head(5)

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


## Rename Columns

In [8]:
class_data=class_data.rename(columns={"ID_Class":"ID_Class__c","ID_Course":"ID_Course__c","Section":"Name",
                                      "StartDate":"StartDate__c","EndDate":"EndDate__c"})

class_data.head(5)

,ID_Class__c,ID_Course__c,Name,StartDate__c,EndDate__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [9]:
class_data["StartDate__c"]=class_data["StartDate__c"].astype(str)
class_data["EndDate__c"]=class_data["EndDate__c"].astype(str)

In [10]:
class_data

,ID_Class__c,ID_Course__c,Name,StartDate__c,EndDate__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


## Convert DataFrame to a list of Dictionaries

In [11]:
class_load = class_data.to_dict('records')
class_load

[{'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Name': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': '2020-03-16',
  'EndDate__c': '2020-03-14'},
 {'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Name': 'GWDC201805DATA3',
  'StartDate__c': '2018-05-15',
  'EndDate__c': '2018-11-08'},
 {'ID_Class__c': 3,
  'ID_Course__c': 2,
  'Name': 'GWARL201905WEB3',
  'StartDate__c': '2019-05-14',
  'EndDate__c': '2019-11-07'},
 {'ID_Class__c': 4,
  'ID_Course__c': 3,
  'Name': 'GWARL201905UIUX3',
  'StartDate__c': '2019-05-14',
  'EndDate__c': '2019-11-07'}]

## Inserting rows in bulk

In [12]:
try:
    sf.bulk.Class__c.insert(class_load)
except Exception as e:
    print(e)